# Bootcamp: Cientista de Dados - Desafio Final

- Data: julho de 2022.

## Autor

Feito por [Alexsander Lopes Camargos](https://github.com/alexcamargos): Entre em contato!

[![GitHub](https://img.shields.io/badge/-AlexCamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=github&logoColor=white&link=https://github.com/alexcamargos)](https://github.com/alexcamargos)
[![Twitter Badge](https://img.shields.io/badge/-@alcamargos-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=twitter&logoColor=white&link=https://twitter.com/alcamargos)](https://twitter.com/alcamargos)
[![Linkedin Badge](https://img.shields.io/badge/-alexcamargos-1ca0f1?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/alexcamargos/)](https://www.linkedin.com/in/alexcamargos/)
[![Gmail Badge](https://img.shields.io/badge/-alcamargos@vivaldi.net-1ca0f1?style=flat-square&labelColor=1ca0f1&logo=Gmail&logoColor=white&link=mailto:alcamargos@vivaldi.net)](mailto:alcamargos@vivaldi.net)

## Licença

[MIT License](https://choosealicense.com/licenses/mit/)

## Objetivos

- Exercitar os seguintes conceitos trabalhados no Bootcamp:
- Pré-processamento dos dados.
- Detecção de anomalias.
- Processamento dos dados.
- Correlações.
- Spark MlLib.
- Interpretação dos dados.

## Enunciado

O derrame é uma das doenças que mais acometem a população mundial. Segundo a World Health Organization (WHO), o Acidente Vascular Cerebral (AVC) foi a segunda maior causa de morte na população mundial no ano de 2016 [https://www.who.int/news-room/fact-sheets/detail/the-top-10-causes-of-death](https://www.who.int/news-room/fact-sheets/detail/the-top-10-causes-of-death).

Neste desafio, vamos realizar uma análise sobre um banco de dados composto por uma pesquisa realizada com diferentes pacientes. Nesta análise, vamos tentar prever, nos baseando em algumas características específicas, se um determinado indivíduo irá ou não sofrer um AVC. Para isso, vamos aplicar o pré-processamento dos dados e a aplicar o modelo de Regressão Logística e o SVM, para indicar se um indivíduo possui ou não maior probabilidade de desenvolver o AVC.

## Atividades

Os alunos deverão desempenhar as seguintes atividades:

1. Acessar o site [https://community.cloud.databricks.com/](https://community.cloud.databricks.com/) e utilizar a conta gratuita. O tutorial de criação da conta gratuita está presente na plataforma Canvas, logo abaixo da apostila, no item “Arquivos complementares” com o nome “tutorial_databricks_TPD.pdf”.
2. Acessar e baixar os arquivos “healthcare_dataset_stroke_data.csv” e o “desafio_CID.ipynb” presentes na pasta: [https://drive.google.com/drive/folders/1alRfzr1WbOVlfvffFo8zrXSlwniW1Nd8?usp=sharing](https://drive.google.com/drive/folders/1alRfzr1WbOVlfvffFo8zrXSlwniW1Nd8?usp=sharing).
3. Responda às perguntas do desafio.

In [ ]:
# Importando os módulos necessários.
from pyspark import SparkFiles
from pyspark.sql import SparkSession

from pyspark.sql import functions as sql_F

In [ ]:
# Criando o SparkSession
spark  = SparkSession.builder.appName('Desafio do Módulo 3').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## Pergunta 1 - Quantas instâncias e características existem, respectivamente, no dataset?

In [ ]:
pass

## Pergunta 2 - Quantas variáveis do tipo “string” estão presentes no dataset?

In [ ]:
pass

## Pergunta 3 - Qual é a idade (age) média dos entrevistados?

In [ ]:
pass

## Pergunta 4 - Sobre a distribuição de AVC em relação ao sexo (gender) dos entrevistados, é CORRETO afirmar:

In [ ]:
pass

## Pergunta 5 - É correto afirmar sobre o dataset, EXCETO:

In [ ]:
pass

## Pergunta 6 - Qual é o valor da mediana para a variável do nível médio de glicose do entrevistado (“avg_glucose_level”)?

In [ ]:
pass

## Pergunta 7 - Analisando o padrão de dispersão da variável do nível médio de glicose do entrevistado (“avg_glucose_level”), é correto afirmar, EXCETO:

In [ ]:
pass

## Pergunta 8 - Analisando a dispersão dos dados para a variável idade (“age”), é correto afirmar, EXCETO:

In [ ]:
pass

## Pergunta 9 - Quantas classes diferentes para a variável “work_type” existem no dataset?

In [ ]:
pass

## Pergunta 10 - Dentre as classes de tipos de trabalhos existentes (work_type), qual é aquela que possui uma maior quantidade de instâncias?

In [ ]:
pass

## Pergunta 11 - Qual foi, respectivamente, o percentual de dados utilizados para o treinamento e teste do modelo?

In [ ]:
pass

## Pergunta 12 - Analisando as variáveis “bmi” e “smoking_status”, é CORRETO afirmar:

In [ ]:
pass

## Pergunta 13 - Após o agrupamento dos dados de “smoking_status” e ”stroke”, é CORRETO afirmar que:

In [ ]:
pass

## Pergunta 14 - Sobre a relação entre a hipertensão (hypertension) e o AVC (stroke) presente neste dataset, é CORRETO afirmar:

In [ ]:
pass

## Pergunta 15 - Sobre o algoritmo de regressão logística aplicado para a previsão da ocorrência de AVC, é correto afirmar, EXCETO:

In [ ]:
pass